# Métricas

Para evaluar los dos modelos creados previamente primero nos centraremos en crear un conjunto de prueba siguiendo los siguientes pasos:

1. Obtener un conjunto de oraciones variadas (C)
    - Oraciones simples
    - Oraciones complejas 
    - Oraciones con ruido

2. TP - que recupere la query tal cual o lo más parecido
    - La query (c in C) si se encuentra en corpus
    - La query se encuentra parcialmente
       (tomamos oraciones del corpus y le metemos ruido)
       (tendríamos que revisar que el corpus si tenga la query parcial
       y luego revisar los resultados, si la busqueda da resultados
       pero en corpus no hay entonces es un falso positivo)


3. TN - oraciones donde ninguna palabra está dentro del corpus
    - Si regresa resultados es un falso negativo 

<p align="center">
  <img src="img/metricas.jpg" width="70%" alt="Metricas"/>
</p>

In [15]:
from sklearn.model_selection import train_test_split #particiones
import string
import random
import pickle
#Para el corpus
import sys
sys.path.append('..')
from src.data import Data 

Cargamos el corpus original para seleccionar algunas oraciones

In [4]:
videos_original = Data("../corpus/data")
videos_original.corpus[0][0]['subtitles'][0]#Nota: no print de todo

{'start': '13.596', 'dur': '1.752', 'text': '¿Adónde vas, Momo?'}

In [25]:
# Obtenemos el conjunto de test
_, test = train_test_split(videos_original.corpus, test_size=0.05)

In [26]:
def get_subtitles():
    '''
    Regresa los subtitulos del corpus
    '''
    test_subtitles = []
    for chanel in test:
        for video in chanel:
            if 'subtitles' in video:
                for s in video['subtitles']:
                    test_subtitles.append(s['text'])
    return test_subtitles

Del 2.5% de oraciones tomadas del corpus tomamos la mitad para meterles ruido

In [27]:
test_TP, test_TPR = train_test_split(get_subtitles(), test_size=0.5)
print(test_TPR[:10], len(test_TPR), len(test_TP))

['estable el rojo de doble hoja escuchen', 'me vea distinto voy a presionar para', 'nos vamos a conectar realmente está', 'sobrepasando los límites tomando riesgos', '[Música]', 'tu espíritu es evidente', 'tiene tanta imaginación como miedo de', 'tal vez ninguno de ellos y te querrá', 'tuve que traer a mi hermano a vivir', '[Música]'] 1814 1814


In [28]:
def insert(s, num):
    """
    Agrega n cadenas aleatorias a una lista

    Args:
        s (list): lista de cadenas
        num (int): número de cadenas aleatorias a insertar
    """
    for _ in range(num):
        noise = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(6))
        ix = random.randint(0,len(s)-1)
        s.insert(ix, noise)

def make_noise():
    """
    A cada oración del conjunto le agrega n cadenas aleatorias
    """
    for i, sentence in enumerate(test_TPR):
        s = sentence.split(" ")
        insert(s, 2)
        test_TPR[i] = ' '.join(s)

make_noise()
print(test_TPR[:10], len(test_TPR))

['estable el Krpvai rojo de doble hoja Yiq09t escuchen', 'me vea distinto YM9WvL voy a WmXDRl presionar para', 'nos vamos FvWJCa CuwjNU a conectar realmente está', 'WcKt0L sobrepasando los límites sQ7Bre tomando riesgos', 'lcIm6W tunOy4 [Música]', 'Q7qGDB tu 328igU espíritu es evidente', 'tiene k93N1s tanta h52ltC imaginación como miedo de', 'tal vez ninguno voKmKN de CqymG2 ellos y te querrá', 'tuve que 5by4Qg traer kNRISK a mi hermano a vivir', 'PpbZIn i41K3Y [Música]'] 1814


Generamos 1.25% de oraciones con palabras que no están en el corpus de tal manera que el conjunto de prueba queda compuesto por:
- 1.25% oraciones que se encuentran tal cual en el corpus
- 1.25% oraciones que se encuentran  en el corpus pero con ruido
- 0.625% oraciones que no comparten palabras con el corpus

In [29]:
def make_sentence():
    """
    Crea un conjunto de cadenas aleatorias
    """
    sentence = []
    lg = [3,5,10,20]
    for _ in range(len(test_TP)//2):
        s = [""]
        insert(s, lg[random.randint(0,len(lg)-1)])
        sentence.append(' '.join(s))
    return sentence
    
test_TN = make_sentence()
print(test_TN[:10], len(test_TN))

['aDn0iN 2yFbJ1 iYySec UIV2nH WvbwKm ', 'DIm6hY lwDrf1 mAGUp5 ', 'zLPNEn e2KMX2 lWmPpC RwpiHX 0mNqTY j60FLh CMoPON p7aifF bWuCLr 9eQXHu ', 'HBu9cb DnDiVN DJUQTT s9DINu mEeRUJ 4FEUKw fmPMtA x2nrcC gOsFXC P8W64E ', 'KyEcMj I9iWB1 jc6Td1 ', 'Y0xPOl VjFXvb L26OJ7 KvKiYO Or9KJB Esz882 8PrcRL DNk53s HMfqzN FpueVf 2tnASF 2lR8cI 8Ls1w4 P3wUkM zbz3Qe SlNbra eQY1NG H7QlCe fLCJc9 VVpYVg ', 'E3Xyux yhqYwK 24tzkl ', 'PNjtxM AjP2HI liE5zy ', 'gAyhM4 7o64aj 5ONjIw G4q3GD NbU0k9 ', 'd4Uzpx KN4izO lfKZpu '] 907


Guardamos el conjunto de prueba

In [30]:
test_all = test_TP + test_TPR + test_TN
pickle.dump(test_all, open("../pkl/test.pkl", "wb"))

In [2]:
with open("../pkl/test.pkl", "rb") as f:
    test_all = pickle.load(f)

test_all[3000:3050]

['blJsis 8eaxn6 asco',
 'l0nrhs LEt41H [Aplausos]',
 'NRKCrq vida como GZRafD detective aquí es donde vivo a',
 'zpty5d LCNCRi más o menos',
 'z1IRhf voy a comerte ZSquts vivo',
 'vamos a celebrar con una gran fiesta qW7Kal RHrYMU un',
 'estos son más CK4oNF que objetos V90QM1 y vestuarios',
 '15 mil 5cFBMT dólares Y8oMHy por favor sin el lado',
 'mundo la apertura del concierto estará O9xVT4 fmqnb9 a',
 'deberíamos 1GdCyN tratar de jFROLG averiguar dónde',
 'Hz9q70 LxbqQR [Aplausos]',
 'aparte están dSdqqk aEB6z2 las personas que desean',
 'VAzBvh n96dw0 [Música]',
 'FQzQ69 uh3qPs [Música]',
 'caminan a 8goHyq fGBiAu madrid aplausos',
 'OeigtJ maravilloso su reto será QGywUK el lograr con',
 'X8Q3qV también tienen momentos s7c3OF turbios cuánto',
 'fiE4Z6 6zxctO estuviste cerca',
 'ella buscar a 5eKhqJ mi hermano y 6OeY4D tú vengas',
 'su rShTIy i4yoCk cabeza verdad la mía es la cultura yo',
 'ard0UB tienen fJX1b2 superpoderes somos superhéroes',
 'el océano te IPtBNu eligió por soJ17

In [3]:
len(test_all)

4535

Ahora veremos que tan bien recupera las oraciones de prueba

In [16]:
def count_words(s1, s2):
    """
    Regresa el número de palabras que conparten 2 cadenas

    Args:
        s1 (lst): lista de palabras de la cadena 1
        s2 (lst): lista de palabras de la cadena 2
    """
    val = 0
    for i in s2: 
        if i in s1: val += 1
    return val

In [17]:
count_words("a un lado oigan alejen a los demás viene".split(), 'y fue porque íbamos a saltar de los'.split())

2

In [18]:
#Para el caso de TFIDF cuando no encuentra coincidencias
#(score = 0 para todas las cadenas)regresa lo siguiente:
dummy_sentence = ['tu papá tuvo un accidente', 'te recuerdo que está', 
                  'aquí', 'estaba como rabioso con', 
                  'especial con lucas martín']

def eval_sentence(query, resp):
    """
    Evalúa los resultados obtenidos dada una query

    Args:
        query (str): query de prueba generada previamente
        resp (list): lista con las primeras 5 respuestas obtenidas 
            por algún mecanismo de RI
    Return:
        (int): 0 cuando no encontró ninguna coincidencia
               1 si encontró la query exacta y es el primer resultado
               (0 - 1) si:
                - Encuentra la query pero no es el primer resultado
                le quitamos .1 por cada lugar que pasa y no está la query
                - No se encuentra la query pero algunas palabras de la misma sí 
    """
    val = 0 
    total = 0

    if resp != dummy_sentence: 
        count = 0
        for i, r in enumerate(resp):
            if r == query: #si la encuentra val será como min .6
                return  1 - (.1 * i)
            total += len(r)
            count += count_words(query.split(), r.split())
            # si no la encuentra ve que tanto se parecen los resultados
            val = count / total 
    return val

Probamos los 4 casos

In [19]:
eval_sentence("avergonzado lo suficiente no apenas",
             ['avergonzado lo suficiente no apenas', 
             'pesados o polar no lo suficiente como', 
             'no vende suficiente leña lo siento pero', 
             'Como pueden ver, no me cuesta trabajo \nestirar la masa porque descanso lo suficiente.', 
             'aún así no es suficiente porque lo'])

1.0

In [20]:
eval_sentence("avergonzado lo suficiente no apenas",
             ['pesados o polar no lo suficiente como', 
             'no vende suficiente leña lo siento pero', 
             'avergonzado lo suficiente no apenas', 
             'Como pueden ver, no me cuesta trabajo \nestirar la masa porque descanso lo suficiente.', 
             'aún así no es suficiente porque lo'])

0.8

In [21]:
eval_sentence("a un lado oigan alejen a los demás viene",   
             ['Vámonos a casa.', 
             '¿Cómo ibas a saberlo?', 
             '¡Hola a todos!', 
             'Son fisuras\na un infierno desconocido.', 
             'y fue porque íbamos a saltar de los'])

0.056910569105691054

In [22]:
eval_sentence("8eIqyc xwWsUF 3eHwS8 qE5XFM Tknsyj",
             ['tu papá tuvo un accidente', 
             'te recuerdo que está', 
             'aquí', 
             'estaba como rabioso con', 
             'especial con lucas martín'])

0

In [23]:
def eval_all(resp):
    """
    Evalua las respuestas de un modelo de la siguiente manera:
        Si está en el conjunto 1 (original) debería de responder con 1
            Si 1 o muy cerca de 1 TP++ else si 0 FN++
        Si esta en el conjunto 2 (con ruido) debería responder con un número decimal
            Si mayor a .5 TP++ else si menor a 5 FN++
        Si está en el conjunto 3 (que no están) deberia responder con 0
            Si 0 TN++ else si 1 FP++
    
    Args:
        resp (list): cada entrada de la lista es una lista con 5 respuestas
    """
    original_noise = zip(test[:3628], resp[:3628])
    invent = zip(test[3629:-1], resp[3629:-1])
    TP, FN, FP, TN = 0, 0, 0, 0
    #Recuperamos las particiones
    #Conjunto original y con ruido
    for query, resp in original_noise:
        r = eval_sentence(query, resp)
        if r > 0.4:
            TP += 1
        else:
            FN +=1
    #Conjunto inventado
    for query, resp in invent:
        r = eval_sentence(query, resp)
        if r == 0:
            TN += 1
        else:
            FP += 1
    return TP, FN, FP, TN 

In [24]:
def get_metrics(TP, FN, FP, TN):
    """
    Realiza el cálculo de:
    Sensitive, specificity, precision, 
    negative predictive value y accuracy 

    Args:
        TP (int): True Positive
        FN (int): False Negative
        FP (int): False Positive
        TN (int): True Negative
    """
    pred = TP / (TP + FP)
    rec =  TP / (TP + FN)
    return {"TP": TP, "FN": FN, "FP": FP, "TN": TN,
            "Specificity": TN / (TN + FP),
            "Negative predictive value": TN / (TN + FN),
            "Precision": pred,
            "Sensitive (recall)": rec,
            "Accuracy": (TP + TN) / (TP + TN + FP + FN),
            "F1": 2*(pred * rec) / (pred + rec)}

In [25]:
with open("../pkl/test.pkl", "rb") as f:
    test = pickle.load(f)

Evaluamos TF-IDF con videos por vectores

In [26]:
with open("../pkl/results.pkl", "rb") as f:
    test_results = pickle.load(f)

TP, FN, FP, TN = eval_all(test_results)
get_metrics(TP, FN, FP, TN)

{'TP': 529,
 'FN': 3099,
 'FP': 0,
 'TN': 905,
 'Specificity': 1.0,
 'Negative predictive value': 0.22602397602397603,
 'Precision': 1.0,
 'Sensitive (recall)': 0.1458103638368247,
 'Accuracy': 0.31634679020516215,
 'F1': 0.2545104642771229}

Evaluamos TF-IDF con videos por subtítulo

In [27]:
with open("../pkl/resultsxOracion.pkl", "rb") as f:
    test_resultsxO = pickle.load(f)

TP, FN, FP, TN = eval_all(test_resultsxO)
get_metrics(TP, FN, FP, TN)

{'TP': 1093,
 'FN': 2535,
 'FP': 0,
 'TN': 905,
 'Specificity': 1.0,
 'Negative predictive value': 0.26308139534883723,
 'Precision': 1.0,
 'Sensitive (recall)': 0.3012679162072767,
 'Accuracy': 0.44076770350761085,
 'F1': 0.46303749205676764}